<a href="https://colab.research.google.com/github/AurelienCD/Formation_OCR_Ing_Machine_Learning/blob/main/LibAUC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importation des librairies

In [135]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [136]:
%cd drive/MyDrive/Formation_OCR_Ing_Machine_Learning/Projet 7/Stage_Noémie/LibAUC

[Errno 2] No such file or directory: 'drive/MyDrive/Formation_OCR_Ing_Machine_Learning/Projet 7/Stage_Noémie/LibAUC'
/content/drive/MyDrive/Formation_OCR_Ing_Machine_Learning/Projet 7/Stage_Noémie/LibAUC


In [1]:
pip install libauc

     |████████████████████████████████| 462 kB 7.1 MB/s 


In [2]:
import pandas as pad 
import numpy as np
from numpy.core.numeric import NaN

import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import statistics
from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import ttest_ind


import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp as multi 

import pandas as pad 
import numpy as np
from numpy.core.numeric import NaN

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics.cluster import adjusted_rand_score

from joblib import dump, load

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV

from sklearn.preprocessing import *
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.metrics import *
from sklearn.model_selection import KFold
from sklearn.metrics import  make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve

import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
import random
import numpy

import pandas.util.testing as tm

import joblib

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from libauc.datasets import imbalance_generator

In [38]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np

# Préparation des données

## Importation des données

In [137]:
df = pad.read_excel('Database_LibAUC.xlsx')
df.dropna(axis = 0, how ='any')

,Label_1,Label_2,Label_3,Label_4,Label_5,Label_6,Class_to_predict
0,0.4633,0.1076,40.4710,0.0859,0.1338,0.8025,1.0
1,0.4540,0.0997,36.4405,0.0815,0.1233,0.7998,1.0
2,0.4880,0.0803,61.0115,0.0558,0.1058,0.7631,1.0
3,0.4113,0.1118,34.3492,0.0926,0.1467,0.7588,1.0
4,0.6148,0.0940,31.4994,0.0792,0.1329,0.7026,1.0
...,...,...,...,...,...,...,...
1627,0.5041,0.1099,46.2522,0.0845,0.1460,0.7514,1.0
1628,0.4571,0.1253,41.3130,0.0995,0.1825,0.6850,1.0
1629,0.3610,0.1406,30.2030,0.1194,0.2063,0.6746,1.0
1630,0.5300,0.0692,64.1103,0.0470,0.0923,0.7494,1.0


In [142]:
# Suppression des données non utiles 
list_to_drop = ['Class_to_predict']
df2 = df.drop(list_to_drop, axis=1)

# Importation des données
X = df2
y = df["Class_to_predict"]

In [93]:
x_data.shape

(1632, 6)

In [145]:
X_np = np.array(X)

In [146]:
sc = StandardScaler()

In [147]:
X_norm = sc.fit_transform(X_np)

In [148]:
x_data = X_norm
y_data = y

In [149]:
le = LabelEncoder()
y_data = np.array(le.fit_transform(y_data))
print(le.classes_)

[0. 1.]


In [173]:
X_train,X_test,y_train,y_test = train_test_split(x_data,y_data,test_size = 0.3,random_state = 0)

## Gestion du unbalanced data (en utilisant la fonction de LibAUC)

In [174]:
SEED = 123
imratio = 0.1

In [175]:
(X_train, y_train) = imbalance_generator(X_train, y_train, imratio=imratio, shuffle=True, random_seed=SEED)
(X_test, y_test) = imbalance_generator(X_test, y_test, is_balanced=True, random_seed=SEED)

NUM_SAMPLES: [117], POS:NEG: [11 : 106], POS_RATIO: 0.0940
NUM_SAMPLES: [490], POS:NEG: [455 : 35], POS_RATIO: 0.9286


# Création du modèle

In [176]:
trainloader = DataLoader((X_train, y_train), batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
testloader = DataLoader((X_test, y_test), batch_size=16, shuffle=False, num_workers=2,  pin_memory=True)

In [165]:
BATCH_SIZE = 16
lr = 0.1
gamma = 500
weight_decay = 1e-4
margin = 1.0

In [166]:
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG
from libauc.models import ResNet20

In [167]:
# You need to include sigmoid activation in the last layer for any customized models!
model = ResNet20(pretrained=False, last_activation='sigmoid', num_classes=1)
#model = model.cuda()

Loss = AUCMLoss(imratio=imratio)
optimizer = PESG(model, 
                 a=Loss.a, 
                 b=Loss.b, 
                 alpha=Loss.alpha, 
                 imratio=imratio, 
                 lr=lr, 
                 gamma=gamma, 
                 margin=margin, 
                 weight_decay=weight_decay)

In [177]:
print ('Start Training')
print ('-'*30)
for epoch in range(100):
    
     if epoch == 50 or epoch==75:
         # decrease learning rate by 10x & update regularizer
         optimizer.update_regularizer(decay_factor=10)
   
     train_pred = []
     train_true = []
     model.train()    
     for data, targets in trainloader:
         data, targets  = data.cuda(), targets.cuda()
         y_pred = model(data)
         loss = Loss(y_pred, targets)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
        
         train_pred.append(y_pred.cpu().detach().numpy())
         train_true.append(targets.cpu().detach().numpy())

     train_true = np.concatenate(train_true)
     train_pred = np.concatenate(train_pred)
     train_auc = roc_auc_score(train_true, train_pred) 

     model.eval()
     test_pred = []
     test_true = [] 
     for j, data in enumerate(testloader):
         test_data, test_targets = data
         test_data = test_data.cuda()
         y_pred = model(test_data)
         test_pred.append(y_pred.cpu().detach().numpy())
         test_true.append(test_targets.numpy())
     test_true = np.concatenate(test_true)
     test_pred = np.concatenate(test_pred)
     val_auc =  roc_auc_score(test_true, test_pred) 
     model.train()
   
     # print results
     print("epoch: {}, train_loss: {:4f}, train_auc:{:4f}, test_auc:{:4f}, lr:{:4f}".format(epoch, loss.item(), train_auc, val_auc, optimizer.lr ))

Start Training
------------------------------


RuntimeError: ignored